In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Загрузка данных MNIST
(train_imgs, train_lbls), (test_images, test_lbls) = mnist.load_data()

# Нормализация данных
train_images = train_imgs / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_lbls = to_categorical(train_lbls)
test_lbls = to_categorical(test_lbls)

# Модель 1: Простая полносвязная нейронная сеть
mlp_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
# Компиляция модели
mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
mlp_model.fit(train_imgs, train_lbls, epochs=5)

# Сохранение модели
mlp_model.save('mlp_mnist.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
# Компиляция модели
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
cnn_model.fit(train_imgs[..., np.newaxis], train_lbls, epochs=5)

# Сохранение модели
cnn_model.save('cnn_mnist.h5')

# Функция FGSM атаки
def fgsm(image, epsilon, gradient):
    perturbed = image + epsilon * np.sign(gradient)
    return np.clip(perturbed, 0, 1)

# Генерация противоречивых примеров для первой модели
def create_adversarial_sample(model, img, lbl, epsilon):
    img_tensor = tf.convert_to_tensor(img.reshape(1, 28, 28, 1))
    lbl_tensor = tf.convert_to_tensor(lbl if len(lbl.shape) == 1 else np.argmax(lbl))

# Вычисление градиента
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        prediction = model(img_tensor)
        loss = tf.keras.losses.categorical_crossentropy(lbl_tensor[None], prediction)

    gradient = tape.gradient(loss, img_tensor)
    return fgsm(img_tensor.numpy(), epsilon, gradient.numpy()).reshape(28, 28, 1)

def create_adversarial_set(model, images, labels, epsilon):
    adversarial_set = []
    for img, lbl in zip(images, labels):
        adv_img = create_adversarial_sample(model, img, lbl, epsilon)
        adversarial_set.append(adv_img.reshape(28, 28))

    return np.array(adversarial_set)

# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adv_imgs_mlp = create_adversarial_set(mlp_model, test_images, test_lbls, epsilon)

# Оценка первой модели на противоречивых примерах
mlp_loss, mlp_acc = mlp_model.evaluate(adv_imgs_mlp, test_lbls, verbose=0)
print(f"MLP точность на противоречивых данных: {mlp_acc}")

# Оценка второй модели на противоречивых примерах (перенос атаки)
adv_imgs_mlp_cnn = adv_imgs_mlp[..., np.newaxis]
cnn_loss, cnn_acc = cnn_model.evaluate(adv_imgs_mlp_cnn, test_lbls, verbose=0)
print(f"CNN точность на противоречивых данных MLP: {cnn_acc}")

# Генерация противоречивых примеров для второй модели
adv_imgs_cnn = create_adversarial_set(cnn_model, test_images[..., np.newaxis], test_lbls, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
adv_imgs_cnn_mlp = adv_imgs_cnn.reshape(-1, 28, 28)
mlp_loss_cnn, mlp_acc_cnn = mlp_model.evaluate(adv_imgs_cnn_mlp, test_lbls, verbose=0)
print(f"MLP точность на противоречивых данных CNN: {mlp_acc_cnn}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8826 - loss: 0.4222
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9630 - loss: 0.1247
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9769 - loss: 0.0762
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9833 - loss: 0.0560
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9872 - loss: 0.0433


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9115 - loss: 0.2973
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 23ms/step - accuracy: 0.9841 - loss: 0.0539
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 24ms/step - accuracy: 0.9902 - loss: 0.0296
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 23ms/step - accuracy: 0.9942 - loss: 0.0183
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9959 - loss: 0.0132


MLP точность на противоречивых данных: 0.10140000283718109
CNN точность на противоречивых данных MLP: 0.9641000032424927
MLP точность на противоречивых данных CNN: 0.9366999864578247


Вывод: анализ показал, что первая модель, для которой были сгенерированы примеры методом FGSM, продемонстрировала значительное снижение точности, что указывает на её высокую уязвимость к такой атаке. В то же время вторая модель оказалась более устойчивой: её точность снизилась незначительно, что свидетельствует о меньшей восприимчивости к переносу атак, разработанных для другой модели.